In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import display, HTML
import os
import re

# Configuration
headers = {"User-Agent": "Your Name your@email.com"}  # Replace with your details
output_folder = r"D:\SEC_HTML_Downloads"
os.makedirs(output_folder, exist_ok=True)

# Company database (Company Name: CIK mapping)
COMPANIES = {
    "SciQuest Inc": "1082526",
    "MedQuist Holdings Inc": "1441567",
    "Angie's List Inc": "1491778",
    "Puma Biotechnology Inc": "1401667",
    "Xplore Technologies Corp": "1177845",
    "Imprimis Pharmaceuticals Inc": "1360214",
    "Micronet Enertec Tech Inc": "854800",
    "XG Technology Inc": "1565228",
    "Cancer Genetics Inc": "1349929",
    "Ringcentral Inc": "1384905",
    "Pioneer Power Solutions Inc": "1449792",
    "Akers Biosciences Inc": "1321834",
    "Intra-Cellular Therapies Inc": "1567514",
    "Malibu Boats Inc": "1590976",
    "Retrophin Inc": "1438533",
    "SMTP Inc": "1506439",
    "Ignyta Inc": "1557421",
    "Sysorex Global Holdings Corp": "1529113",
    "Pacific Datavision Inc": "1304492"
}

def get_filings(cik, company_name):
    """Get all S-1 filings for a company using SEC website"""
    url = f"https://www.sec.gov/cgi-bin/browse-edgar?CIK={cik}&action=getcompany&type=S-1&count=100"
    try:
        response = requests.get(url, headers=headers, timeout=30)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        filings = []
        table = soup.find('table', {'class': 'tableFile2'})
        if not table:
            return filings
            
        for row in table.find_all('tr')[1:]:  # Skip header row
            cells = row.find_all('td')
            if len(cells) >= 4:
                form_type = cells[0].text.strip()
                if form_type in ['S-1', 'S-1/A']:
                    filing_date = cells[3].text.strip()
                    doc_link = cells[1].find('a', {'id': 'documentsbutton'})
                    if doc_link:
                        href = doc_link['href']
                        accession_no = re.search(r'/(\d{10}-\d{2}-\d{6})', href)
                        if accession_no:
                            filings.append({
                                'form': form_type,
                                'filingDate': filing_date,
                                'accessionNumber': accession_no.group(1)
                            })
        return filings
    except Exception as e:
        print(f"⚠️ Error processing {company_name}: {str(e)}")
        return []

def get_primary_doc_url(cik, accession_no):
    """Get the primary document URL from the filing index page"""
    acc_no_clean = accession_no.replace("-", "")
    index_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_no_clean}/{accession_no}-index.html"
    
    try:
        response = requests.get(index_url, headers=headers, timeout=30)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the primary document (first HTML file)
        for row in soup.select('tr'):
            if row.find('a'):
                filename = row.find('a').text
                if filename.lower().endswith(('.htm', '.html')):
                    return f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_no_clean}/{filename}"
        
        return index_url
    
    except Exception as e:
        print(f"⚠️ Error retrieving document: {str(e)}")
        return index_url

# Main execution
print("Starting SEC filings extraction...\n")
results = []

for company_name, cik in COMPANIES.items():
    print(f"Processing {company_name}...")
    
    filings = get_filings(cik, company_name)
    
    if not filings:
        print(f"⚠️ No S-1 filings found for {company_name}")
        continue
    
    for filing in filings:
        doc_url = get_primary_doc_url(cik, filing['accessionNumber'])
        results.append({
            "Form Type": filing['form'],
            "Filing Date": filing['filingDate'],
            "Company": company_name,
            "CIK": cik,
            "SEC Link": doc_url,
            "View Filing": f'=HYPERLINK("{doc_url}", "View Filing")'
        })

# Create DataFrame with exact column order
columns_order = ["Form Type", "Filing Date", "Company", "CIK", "SEC Link", "View Filing"]
all_results = pd.DataFrame(results, columns=columns_order)

# Save and display results
if not all_results.empty:
    csv_path = os.path.join(output_folder, "SEC_Filings_Output.csv")
    try:
        if os.path.exists(csv_path):
            try:
                os.remove(csv_path)
            except PermissionError:
                print(f"\n❌ Please close the file if open in Excel: {csv_path}")
                raise
        
        all_results.to_csv(csv_path, index=False)
        print(f"\n✅ All filings saved to: {csv_path}")
        
        # Display results in desired format
        display(HTML(
            all_results.to_html(escape=False, index=False)
        ))
    except Exception as e:
        print(f"\n❌ Error saving file: {str(e)}")
else:
    print("\nNo filings were processed. Please check for errors above.")

print("\nProcessing complete!")

Starting SEC filings extraction...

Processing SciQuest Inc...
Processing MedQuist Holdings Inc...
Processing Angie's List Inc...
Processing Puma Biotechnology Inc...
Processing Xplore Technologies Corp...
Processing Imprimis Pharmaceuticals Inc...
Processing Micronet Enertec Tech Inc...
Processing XG Technology Inc...
Processing Cancer Genetics Inc...
⚠️ Error retrieving document: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Processing Ringcentral Inc...
Processing Pioneer Power Solutions Inc...
